SPOTIFY EDA

In [1]:
import pandas as pd
import numpy as np

import spotipy
import json
import csv
from spotipy.oauth2 import SpotifyClientCredentials

I will first load the data_by_artist.csv to a dataframe

In [6]:
artist_df = pd.read_csv('songs_dataset/data_by_artist.csv')
artist_df.describe()

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
count,27621.000000,27621.000000,2.762100e+04,27621.000000,27621.000000,27621.000000,27621.000000,27621.000000,27621.000000,27621.000000,27621.000000,27621.000000,27621.000000,27621.000000
mean,0.504651,0.544411,2.374886e+05,0.493708,0.166661,0.203603,-11.222480,0.096180,115.439232,0.516032,34.288478,5.409942,0.767170,14.313493
std,0.369788,0.176098,1.209766e+05,0.253566,0.292133,0.140311,5.819377,0.116062,25.138470,0.243304,22.104194,3.485676,0.422643,53.867027
min,0.000000,0.000000,1.879550e+04,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.128784,0.429000,1.819730e+05,0.282000,0.000004,0.111667,-14.097000,0.039500,98.725500,0.334000,13.511111,2.000000,1.000000,2.000000
50%,0.493000,0.555667,2.176930e+05,0.501000,0.001610,0.162850,-10.135333,0.052644,114.950000,0.528500,39.000000,6.000000,1.000000,3.000000
75%,0.901000,0.673000,2.660000e+05,0.699000,0.188000,0.247371,-6.906000,0.098000,129.612647,0.705000,51.000000,8.000000,1.000000,9.000000
max,0.996000,0.986000,5.403500e+06,1.000000,1.000000,0.991000,1.342000,0.964000,217.743000,0.991000,95.000000,11.000000,1.000000,3172.000000


In [2]:
# INITIALIZE SPOTIFY API CREDS
credentials = json.load(open('spotify_auth/authorization.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']

client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [8]:
no_result = []
artist_id_list = []

def search_artist(artist, market=None):
    search_result = sp.search(artist, limit=20, offset=0, type='artist', market=market)

    if not search_result['artists']['items']:
        print('NO RESULT FOUND', artist) 
        no_result.append(artist)
        return
    for index in range(0,len(search_result['artists']['items'])):
        result = search_result['artists']['items'][index]
        if result['name'].lower() == artist.lower():
            print('RESULT FOUND')
            artist = {'ID': search_result['artists']['items'][index]['id'], 'ARTIST': artist}
            artist_id_list.append(artist)
            break
        if index == len(search_result['artists']['items']) - 1:
            no_result.append(artist)
            print('NO RESULT FOUND', artist)

In [ ]:
for artist in artist_df['artists']:
    search_artist(artist)

I saved the id of artist on a csv so I don't have to run the method multiple times

In [ ]:
with open('artist_id.csv', mode='w', encoding='utf-8') as csv_file:
    field_names = ['ID', 'ARTIST']
    writer = csv.DictWriter(csv_file, fieldnames=field_names)
    writer.writeheader()
    writer.writerows(artist_id_list)
        

In [9]:
len(no_result)
# 27468 

27468

In [10]:
len(artist_id_list)

153

There are 153 artist that the search method from API did not find. I tried to manually search them on gooogle and on the spotify app and some did not appear, some had multiple response some did not even have uploaded songs. Since this data is for personal use only I decided to skip this for now and will try to solve this issue in the future.

In [5]:
artist_id_df = pd.read_csv('artist_id.csv')
artist_id_df

,ID,ARTIST
0,5PwgSnrvg48DYvdsf2Y2ZD,"""Cats"" 1981 Original London Cast"
1,2cez574BwRD00PGEbNl4SH,"""Cats"" 1983 Broadway Cast"
2,2vIKM7x5kKzKR9rIMBejxr,"""Fiddler On The Roof” Motion Picture Chorus"
3,0yultMio5TntZCl8JRRibP,"""Fiddler On The Roof” Motion Picture Orchestra"
4,7CD4S5LN8BeM1MVjL7ASK4,"""Joseph And The Amazing Technicolor Dreamcoat""..."
...,...,...
27463,13ILSgBPHS5T6YCFw9Pc7R,鳳飛飛
27464,22eZSsFE2fxLnnC9Zga25b,黃品源
27465,0AZ5FjdvYk3mKDw0B16nbv,黃國隆
27466,4SDgIGeqg4jyQOZVFWATAF,黃蜀娟


After careful observation, some local artists in my country are missing from the list provided by the dataset. So after a few quick searches I retreived all the names of the local bands and artists
 in PH and now I have to search for them and retrieve their respective spotify IDs

In [ ]:
local_bands = open("songs_dataset/local_bands.txt", encoding='utf-8').read().split("\n")
# local_bands

In [ ]:
local_singers = open("songs_dataset/local_singers.txt", encoding='utf-8').read().split("\n")
# local_singers

In [ ]:
no_result = []
artist_id_list = []

In [ ]:
for artist in local_bands:
    search_artist(artist, 'PH')

In [ ]:
for artist in local_singers:
    search_artist(artist, 'PH')

I added a second parameter that would specify what market they are avaiable. I made this so that I can ensure that they are the local artist and that I won't get a band or an artist from another country

In [11]:
len(no_result)

60

Since there is ony 60 artist with no resuts, I took the time if I can manually search for their spotify pages. I found out that most of the artist don't use their whole names for their spotify pages and the others just don't have a spotify page.

Below are the IDs of the artist that I manually searched for.


In [ ]:
artist_id_list.append({'ID': '4XY9rlr5TDyTuRLP1kbs9F', 'ARTIST': 'Mak and the Dudes'})
artist_id_list.append({'ID': '1wNHRv83hRynWs1Lbn02D0', 'ARTIST': 'Juan de la Cruz Band'})
artist_id_list.append({'ID': '4dlsoFRsNu2hApC2vQSx7b', 'ARTIST': 'Oktaves'})
artist_id_list.append({'ID': '4wW95hzMUSb1zBs34qjLir', 'ARTIST': 'Johnny Alegre Affinity'})
artist_id_list.append({'ID': '2Qa5HCTS7IILwgNlhQILbu', 'ARTIST': 'Marcus Adoro'})
artist_id_list.append({'ID': '62WbvkXqQGvXQvw74GU3kQ', 'ARTIST': 'Morissette Amon'})
artist_id_list.append({'ID': '4h5syho7OXuY0whQVQs1x3', 'ARTIST': 'Junji Arias'})
artist_id_list.append({'ID': '0lslA51eKXwTUY0TSX9Emy', 'ARTIST': 'Marion Aunor'})
artist_id_list.append({'ID': '74HXAldpivlRnEqr00aD0T', 'ARTIST': 'Heber Bartolome'})
artist_id_list.append({'ID': '2hJUlesr1UOWuwTKaliasp', 'ARTIST': 'Jay Anne Bautista'})
artist_id_list.append({'ID': '7BxbcnOfx5r4d53UQl2I1s', 'ARTIST': 'Zild Benitez'})
artist_id_list.append({'ID': '2wijJ8u0oAhCehR0MbHIF9', 'ARTIST': 'Acel Bisa'})
artist_id_list.append({'ID': '6ToUesXACuu1mtV4HFcoG3', 'ARTIST': 'Chacha Cañete'})
artist_id_list.append({'ID': '0Q484FO1YILS6Iw0UWATFf', 'ARTIST': 'Joyce Ching'})
artist_id_list.append({'ID': '6qcfs3Wo3SAWE0INK5FDbT', 'ARTIST': 'Noel Comia Jr.'})
artist_id_list.append({'ID': '6jH7RAA1CW3vVj1a312h3Z', 'ARTIST': 'Curtismith'})
artist_id_list.append({'ID': '5ojBrHoZb7pCOcAdEDOoNW', 'ARTIST': 'Glaiza De Castro'})
artist_id_list.append({'ID': '74KlZuH1AECATYKwBlWAdg', 'ARTIST': 'Jason Dy'})
artist_id_list.append({'ID': '53qa1XIZ6pZuhrGDetCGew', 'ARTIST': 'Kyle Echarri'})
artist_id_list.append({'ID': '6UIEzOUxN61pdpQrAmVKDm', 'ARTIST': 'Heart Evangelista'})
artist_id_list.append({'ID': '4910QV0gSyS5yhcRRDNLcL', 'ARTIST': 'Larry Gee'})
artist_id_list.append({'ID': '1n9pyfNaKCPPOi6I3aKnkn', 'ARTIST': 'RJ Jimenez'})
artist_id_list.append({'ID': '3D5DKmCiZ3VQRu68k87E3Q', 'ARTIST': 'Lala Karmela'})
artist_id_list.append({'ID': '2vTbuBRo7ACOZ3JsCnaL7S', 'ARTIST': 'Kyla'})
artist_id_list.append({'ID': '0a4r2EnsevvHCukoJ1xFwJ', 'ARTIST': 'Juan Karlos Labajo'})
artist_id_list.append({'ID': '1PXQjLTz73UtuLmEAWaZGf', 'ARTIST': 'Maloy Lozanes'})
artist_id_list.append({'ID': '17Do37U0HEbxu1tOrKH8dL', 'ARTIST': 'Nadine Lustre'})
artist_id_list.append({'ID': '7DRgc0ko15aNa7DMeZ3J8G', 'ARTIST': 'Jed Madela'})
artist_id_list.append({'ID': '0c7d9V8rjfT6tzojfBfRf5', 'ARTIST': 'Bamboo'})
artist_id_list.append({'ID': '5relTuuSSW001zewisaAk2', 'ARTIST': 'Raymond Manalo'})
artist_id_list.append({'ID': '1onz2fxn4zwyarm9FPVuyp', 'ARTIST': 'Miko Manguba'})
artist_id_list.append({'ID': '7fW1bbURcl5aOUvrCj4va2', 'ARTIST': 'Sophia Montecarlo'})
artist_id_list.append({'ID': '4OjU5UP0GFWeniBC82sGBY', 'ARTIST': 'Kitchie Nadal'})
artist_id_list.append({'ID': '1t6esaYFDM9fOnvjV31v6r', 'ARTIST': 'Robbie Navarro'})
artist_id_list.append({'ID': '0XOSmu9Qn1agQULUVe0dAK', 'ARTIST': 'Iñigo Pascual'})

In [ ]:
with open('local_artist_id.csv', mode='w', encoding='utf-8') as csv_file:
    field_names = ['ID', 'ARTIST']
    writer = csv.DictWriter(csv_file, fieldnames=field_names)
    writer.writeheader()
    writer.writerows(artist_id_list)

In [4]:
local_artist_id_df = pd.read_csv('local_artist_id.csv')
local_artist_id_df 

,ID,ARTIST
0,5pbdyhkEpMobD6Fs6h729Q,Asin
1,6hwX2mCENcTYJ3ls8DPfYb,Humanfolk
2,4RA4PC3WHM9OHZLExirTcU,APO Hiking Society
3,0NTCLJMe5JtkMt3YQuja45,Boyfriends
4,65QcnpMJFv1yogXwoOe97v,Cinderella
...,...,...
325,7fW1bbURcl5aOUvrCj4va2,Sophia Montecarlo
326,4OjU5UP0GFWeniBC82sGBY,Kitchie Nadal
327,1t6esaYFDM9fOnvjV31v6r,Robbie Navarro
328,0XOSmu9Qn1agQULUVe0dAK,Iñigo Pascual


In [6]:
all_artist_id = pd.concat([artist_id_df, local_artist_id_df])
all_artist_id.to_csv('all_artist.csv', index=False)

In [8]:
all_artist_id = pd.read_csv('all_artist.csv')
all_artist_id.describe()

,ID,ARTIST
count,27798,27798
unique,27732,27756
top,0a4r2EnsevvHCukoJ1xFwJ,Smokey Mountain
freq,2,2


After merging the two dataframes, there are duplicated values, which means that there are some philippine artist already in the dataset. Since they are duplicated, I need to remove them 

In [9]:
all_artist_id = all_artist_id.drop_duplicates(subset=['ID'], keep='first')
all_artist_id.describe()

,ID,ARTIST
count,27732,27732
unique,27732,27729
top,3MWm2G53GJPXxzcEEqzgTW,Arcadia
freq,1,2


Now that I have all the ids of the artist, I want to get all of songs from the albums of the artist that is available in PH.

In [19]:
all_artist_id['ALBUMS'] = np.nan
all_artist_id['ALBUMS'] = all_artist_id['ALBUMS'].astype('object')

In [35]:
# for index, row in all_artist_id.iterrows():
#     result = []
#     print(index)
#     id = row['ID']
#     album = sp.artist_albums(id, album_type='album')
#     album_id = [item['id'] for item in album['items'] if 'PH' in item['available_markets']]
#     print("ALBUM", album_id)
#     result = result + album_id
#     print("SOMETHING", result)

#     album = sp.artist_albums(id, album_type='single')
#     album_id = [item['id'] for item in album['items'] if 'PH' in item['available_markets']]
#     # print("SINGLES", album_id)
#     result = result + album_id

#     album = sp.artist_albums(id, album_type='appears_on')
#     album_id = [item['id'] for item in album['items'] if 'PH' in item['available_markets']]
#     # print("Appears On", album_id)
#     result = result + album_id

#     album = sp.artist_albums(id, album_type='compilation')
#     album_id = [item['id'] for item in album['items'] if 'PH' in item['available_markets']]
#     # print("Compilation On", album_id)
#     result = result + album_id

#     all_artist_id.at[index, 'ALBUMS'] = result

for index in range(25001,27798):
    result = []
    print(index)
    id = all_artist_id.iloc[0]['ID']
    album = sp.artist_albums(id, album_type='album')
    album_id = [item['id'] for item in album['items'] if 'PH' in item['available_markets']]
    result = result + album_id

    album = sp.artist_albums(id, album_type='single')
    album_id = [item['id'] for item in album['items'] if 'PH' in item['available_markets']]
    # print("SINGLES", album_id)
    result = result + album_id

    album = sp.artist_albums(id, album_type='appears_on')
    album_id = [item['id'] for item in album['items'] if 'PH' in item['available_markets']]
    # print("Appears On", album_id)
    result = result + album_id

    album = sp.artist_albums(id, album_type='compilation')
    album_id = [item['id'] for item in album['items'] if 'PH' in item['available_markets']]
    # print("Compilation On", album_id)
    result = result + album_id

    all_artist_id.at[index, 'ALBUMS'] = result

7
21668
21669
21670
21671
21672
21673
21674
21675
21676
21677
21678
21679
21680
21681
21682
21683
21684
21685
21686
21687
21688
21689
21690
21691
21692
21693
21694
21695
21696
21697
21698
21699
21700
21701
21702
21703
21704
21705
21706
21707
21708
21709
21710
21711
21712
21713
21714
21715
21716
21717
21718
21719
21720
21721
21722
21723
21724
21725
21726
21727
21728
21729
21730
21731
21732
21733
21734
21735
21736
21737
21738
21739
21740
21741
21742
21743
21744
21745
21746
21747
21748
21749
21750
21751
21752
21753
21754
21755
21756
21757
21758
21759
21760
21761
21762
21763
21764
21765
21766
21767
21768
21769
21770
21771
21772
21773
21774
21775
21776
21777
21778
21779
21780
21781
21782
21783
21784
21785
21786
21787
21788
21789
21790
21791
21792
21793
21794
21795
21796
21797
21798
21799
21800
21801
21802
21803
21804
21805
21806
21807
21808
21809
21810
21811
21812
21813
21814
21815
21816
21817
21818
21819
21820
21821
21822
21823
21824
21825
21826
21827
21828
21829
21830
21831
21832
21833
21

In [33]:
all_artist_id

,ID,ARTIST,ALBUMS
0,5PwgSnrvg48DYvdsf2Y2ZD,"""Cats"" 1981 Original London Cast","['1B1D8rJ34fIaWNAQ2uQffP', '66UrYsmX0lRzAOwLxk..."
1,2cez574BwRD00PGEbNl4SH,"""Cats"" 1983 Broadway Cast","['1B1D8rJ34fIaWNAQ2uQffP', '66UrYsmX0lRzAOwLxk..."
2,2vIKM7x5kKzKR9rIMBejxr,"""Fiddler On The Roof” Motion Picture Chorus","['1B1D8rJ34fIaWNAQ2uQffP', '66UrYsmX0lRzAOwLxk..."
3,0yultMio5TntZCl8JRRibP,"""Fiddler On The Roof” Motion Picture Orchestra","['1B1D8rJ34fIaWNAQ2uQffP', '66UrYsmX0lRzAOwLxk..."
4,7CD4S5LN8BeM1MVjL7ASK4,"""Joseph And The Amazing Technicolor Dreamcoat""...","['1B1D8rJ34fIaWNAQ2uQffP', '66UrYsmX0lRzAOwLxk..."
...,...,...,...
27793,7fW1bbURcl5aOUvrCj4va2,Sophia Montecarlo,NaN
27794,4OjU5UP0GFWeniBC82sGBY,Kitchie Nadal,NaN
27795,1t6esaYFDM9fOnvjV31v6r,Robbie Navarro,NaN
27796,0XOSmu9Qn1agQULUVe0dAK,Iñigo Pascual,NaN


In [34]:
all_artist_id.to_csv('all_artist_album.csv', index=False)

In [30]:
all_artist__album_id = pd.read_csv('all_artist_album.csv')

In [31]:
all_artist__album_id

,ID,ARTIST,ALBUMS
0,5PwgSnrvg48DYvdsf2Y2ZD,"""Cats"" 1981 Original London Cast","['1B1D8rJ34fIaWNAQ2uQffP', '66UrYsmX0lRzAOwLxk..."
1,2cez574BwRD00PGEbNl4SH,"""Cats"" 1983 Broadway Cast","['1B1D8rJ34fIaWNAQ2uQffP', '66UrYsmX0lRzAOwLxk..."
2,2vIKM7x5kKzKR9rIMBejxr,"""Fiddler On The Roof” Motion Picture Chorus","['1B1D8rJ34fIaWNAQ2uQffP', '66UrYsmX0lRzAOwLxk..."
3,0yultMio5TntZCl8JRRibP,"""Fiddler On The Roof” Motion Picture Orchestra","['1B1D8rJ34fIaWNAQ2uQffP', '66UrYsmX0lRzAOwLxk..."
4,7CD4S5LN8BeM1MVjL7ASK4,"""Joseph And The Amazing Technicolor Dreamcoat""...","['1B1D8rJ34fIaWNAQ2uQffP', '66UrYsmX0lRzAOwLxk..."
...,...,...,...
27793,7fW1bbURcl5aOUvrCj4va2,Sophia Montecarlo,NaN
27794,4OjU5UP0GFWeniBC82sGBY,Kitchie Nadal,NaN
27795,1t6esaYFDM9fOnvjV31v6r,Robbie Navarro,NaN
27796,0XOSmu9Qn1agQULUVe0dAK,Iñigo Pascual,NaN


In [57]:
albums = all_artist_id.iloc[22907]['ALBUMS']
print(albums[0])

[


In [56]:
# x = [sp.album(album)['name'] for album in albums]
len(albums)

740

In [153]:
list = all_artist_id.loc[all_artist_id['ID'] == '5PwgSnrvg48DYvdsf2Y2ZD']['ALBUMS']
for item in list:
    print(item)

['1B1D8rJ34fIaWNAQ2uQffP', '66UrYsmX0lRzAOwLxkPgjl', '6xacl8ZZIFmVGErcUovvhy', '1rKa4RNSiyQBIjZ8ET54DQ', '3qqiudxVW3GmigrTHRUcLf', '6lfGhIVE8by4922uwuCxla']


In [122]:
album = sp.artist_albums(all_artist_id['ID'][0], album_type='album')
album_id = [item['id'] for item in album['items'] if 'PH' in item['available_markets']]
album_id




['1B1D8rJ34fIaWNAQ2uQffP', '66UrYsmX0lRzAOwLxkPgjl']